<a href="https://colab.research.google.com/github/JuanpaQC/Beta01-Project/blob/Master---Updates/Project_Face_RecognitionV2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instituto Tecnológico de Costa Rica
### Reconocimiento de Rostros
Author: [Johan Rodríguez](https://github.com/Johanx22x)

Note: When using this notebook, you must select a GPU Hardware Accelerator instead of CPU. Go to Runtime->Change Runtime Type and select any option that uses GPU.

# Dependencies

In [ ]:
!pip install face_recognition         # Install face_recognition package using pip
import face_recognition
import cv2                            # Import OpenCV
import numpy as np
from matplotlib import pyplot as plt  # pyplot

# ipywidgets packages
import ipywidgets as widgets
from IPython.display import display, clear_output

# Live Streaming dependencies
from IPython.display import Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io
import html
import time

from zmq.sugar.socket import T
from traitlets.config import Container
import pickle
import cryptography
import json
from datetime import datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566171 sha256=e83a7bd230df21163b485fb812bf8bd958904f57a43d0270fef05034c6ed955b
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


# Utils

## In-memory data

In [ ]:
registered_faces = {}
lista_asistencia = []

In [ ]:
student_list = []
teacher_list = []

## face recognition

In [ ]:
def check_if_face_registered(face_encoding):
  for _, registered_face in registered_faces.items():
    if True in face_recognition.compare_faces([face_encoding], registered_face):
      return True
  return False

## Real time streaming

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 800, 600);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 800; //video.videoWidth;
      captureCanvas.height = 600; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

In [ ]:
# JavaScript code to stop the webcam stream
js_stop_stream = """
    const stream = video.srcObject;
    const tracks = stream.getTracks();

    tracks.forEach(function(track) {
        track.stop();
    });

    video.srcObject = null;
    div.style.display = 'none';  // Hide the container
"""

# Function to stop the video streaming
def stop_video_stream():
    display(Javascript(js_stop_stream))

# Register Faces

## Register Teachers

In [ ]:
class People():
  def __init__(self, id, name, date_of_birth, nationallity, phone, email, address):
    self.id = id
    self.name = name
    self.date_of_birth = date_of_birth
    self.nationallity = nationallity
    self.phone = phone
    self.email = email
    self.address = address

In [ ]:
class Teacher(People):
    def __init__(self, id, name, date_of_birth, nationallity, phone, email, address, speciallity):
      People.__init__(self, id, name, date_of_birth, nationallity, phone, email, address)
      self.speciality = speciallity

In [ ]:
# Create a text input widget for the name
teacher_name_input = widgets.Text(
    placeholder='Enter the name',
    description='Name:',
    disabled=False
)

teacher_id_input = widgets.Text(
    placeholder='Enter the ID',
    description='ID:',
    disabled=False
)

teacher_date_of_birth_input = widgets.DatePicker(
  description='Date of birth',
  disabled=False
)

teacher_nationallity_input = widgets.Text(
    placeholder='Enter the Nationallity',
    description='Nationallity:',
    disabled=False
)

teacher_phone_input = widgets.Text(
    placeholder='Enter the Phone Number',
    description='Number:',
    disabled=False
)

teacher_email_input = widgets.Text(
    placeholder='Enter the Email',
    description='Email:',
    disabled=False
)

teacher_address_input = widgets.Text(
    placeholder='Enter the Teacher Address',
    description='Address:',
    disabled=False
)

teacher_speciality_input = widgets.Text(
    placeholder='Enter the Teacher Speciality',
    description='Speciality:',
    disabled=False
)

# Create a file upload widget
upload_button = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='Upload Image'
)

or_text = widgets.HTML(
    value=" or "
)

# Create a button to trigger face registration
teacher_register_button = widgets.Button(
    description='Register Face'
)
teacher_register_button.style.button_color = 'green'

back_button = widgets.Button(
    description="Back to Menu"
)
back_button.style.button_color = 'darkred'

capture_button = widgets.Button(
    description='Capture Image'
)

# Create an output widget to display the uploaded or captured face image
output = widgets.Output()

# Initialize variables to store the uploaded image and name
uploaded_image_data = None
registered_name = None
registered_id = None

In [ ]:
# Define a function to handle image upload
def handle_upload_teacher(change):
  global uploaded_image_data
  uploaded_filename = list(upload_button.value.keys())[0]
  uploaded_image_data = upload_button.value[uploaded_filename]['content']

  with output:
      clear_output()
      display(widgets.Image(value=uploaded_image_data))

  # Assuming uploaded_image_data is binary image data in bytes
  image_array = np.frombuffer(uploaded_image_data, np.uint8)
  image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

  # Convert the image to RGB format
  uploaded_image_data = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
upload_button.observe(handle_upload_teacher, names='value')

In [ ]:
def handle_capture_teacher(button):
  global uploaded_image_data
  # start streaming video from webcam
  video_stream()
  js_reply = video_frame("Capturing...", "")

  stop_video_stream()  # Stop the video stream after capturing the frame

  if not js_reply:
    with output:
      clear_output()
      print("Error capturing image!")

  # convert JS response to OpenCV Image
  uploaded_image_data = js_to_image(js_reply["img"])

  with output:
    clear_output()
    # Encode the captured frame as PNG
    _, buffer = cv2.imencode('.png', uploaded_image_data)

    # Convert the encoded image to bytes
    image_bytes = buffer.tobytes()
    display(widgets.Image(value=image_bytes))

capture_button.on_click(handle_capture_teacher)

In [ ]:
# Define a function to handle face registration
def handle_register_teacher(button):

  if uploaded_image_data is not None:

    #asigment value
    registered_teacher_name = teacher_name_input.value
    registres_teacher_id = teacher_id_input.value

    # Detect faces in the frame
    face_locations = face_recognition.face_locations(uploaded_image_data)

    # If there is no face, raise a popup window
    if len(face_locations) == 0:
      with output:
        clear_output()
        print("No face detected in the image!")
        return

    # If there is more than one face, raise a popup window
    if len(face_locations) > 1:
      with output:
        clear_output()
        print("More than one face detected in the image!")
        return

    # Get the face encodings
    face_encodings = face_recognition.face_encodings(uploaded_image_data)

    # Check if the face is registered
    if check_if_face_registered(face_encodings[0]):
      with output:
        clear_output()
        print("Face already registered!")
        return
    with output:
      clear_output()
      print(f"Face registered for {registered_teacher_name}")
      registered_faces[registered_teacher_name] = face_encodings[0]

    # Nombre del archivo JSON
  archivo_bin = 'LoD_Teacher.bin'

  # Abrir el archivo JSON en modo lectura para cargar los datos existentes
  try:
      with open(archivo_bin, 'rb') as archivo:
          datos_existentes = pickle.load(archivo)
  except FileNotFoundError:
      # Si el archivo no existe, crea una lista vacía
      datos_existentes = []

  maestro = Teacher(
    teacher_id_input.value,
    teacher_name_input.value,
    teacher_date_of_birth_input.value,
    teacher_nationallity_input.value,
    teacher_phone_input.value,
    teacher_email_input.value,
    teacher_address_input.value,
    teacher_speciality_input.value
  )

  # Create a dictionary for teacher information
  teacher_info = {
    'ID': str(maestro.id),
    'Name': str(maestro.name),
    'Date of Birth': str(maestro.date_of_birth),
    'Nationallity': str(maestro.nationallity),
    'Phone':str(maestro.phone),
    'Email':str(maestro.email),
    'Address':str(maestro.address),
    'Speciality':str(maestro.speciality),
    'Image': str(face_encodings[0])
  }

  # Agrega el nuevo diccionario a la lista de datos existente
  datos_existentes.append(teacher_info)
  teacher_list.append(teacher_info['Name','ID'])


  # Abre el archivo JSON en modo escritura y guarda la lista actualizada
  with open(archivo_bin, 'wb') as archivo:
      pickle.dump(datos_existentes, archivo)  # El argumento indent hace que el archivo sea más legible
      print(f"Data successfully registered in {archivo_bin}")

teacher_register_button.on_click(handle_register_teacher)

In [ ]:
# function to go back in the menu
def back_menu(b):
  clear_output()
  dataAdder()
back_button.on_click(back_menu)

# add teachers menu
def add_teacher():
  container = widgets.VBox([teacher_name_input, teacher_id_input, teacher_date_of_birth_input, teacher_nationallity_input, teacher_phone_input, teacher_email_input, teacher_address_input, teacher_speciality_input, widgets.HBox([upload_button, or_text, capture_button]), widgets.HBox([teacher_register_button, back_button]), output])

  html_label = widgets.HTML(
      value='<h1 style="color: white; font-size: 24px;">Add Teacher Details</h1>'
  )

  # Mostrar el widget en la celda de Jupyter Notebook
  display(html_label)
  display(container)

## Register Students

In [ ]:
class Student(People):
  def __init__(self, id, name, date_of_birth, nationallity, phone, email, address, career):
    People.__init__(self, id, name, date_of_birth, nationallity, phone, email, address)
    self.career = career

In [ ]:
# Create a text input widget for the name
student_name_input = widgets.Text(
    placeholder='Enter the name',
    description='Name:',
    disabled=False
)

student_id_input = widgets.Text(
    placeholder='Enter the ID',
    description='ID:',
    disabled=False
)

student_date_of_birth_input = widgets.DatePicker(
  description='Date of birth',
  disabled=False
)

student_nationallity_input = widgets.Text(
    placeholder='Enter the Nationallity',
    description='Nationallity:',
    disabled=False
)

student_phone_input = widgets.Text(
    placeholder='Enter the Phone Number',
    description='Number:',
    disabled=False
)

student_email_input = widgets.Text(
    placeholder='Enter the Email',
    description='Email:',
    disabled=False
)

student_address_input = widgets.Text(
    placeholder='Enter the Student Address',
    description='Address:',
    disabled=False
)

opciones = [
            "Software Engineering",
            "Industrial Production Engineering",
            "Engineering in Agronomy",
            "Business Administration",
            "Electronics Engineering",
            "Computer Engineering",
            "Sustainable Rural Tourism Management",
            "Tourism Sustainability Management",
            ]
student_career_dropdown = widgets.Dropdown(
  options=opciones,
  value=opciones[0],  # Valor inicial
  description='Career'
)

# Create a file upload widget
upload_button = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='Upload Image'
)

or_text = widgets.HTML(
    value=" or "
)

# Create a button to trigger face registration
register_button_student = widgets.Button(
    description='Register Face'
)
register_button_student.style.button_color = 'green'

capture_button = widgets.Button(
    description='Capture Image'
)

# Create an output widget to display the uploaded or captured face image
output = widgets.Output()

# Initialize variables to store the uploaded image and name
uploaded_image_data = None
registered_name = None

In [ ]:
# Define a function to handle image upload
def handle_upload_student(change):
  global uploaded_image_data
  uploaded_filename = list(upload_button.value.keys())[0]
  uploaded_image_data = upload_button.value[uploaded_filename]['content']

  with output:
      clear_output()
      display(widgets.Image(value=uploaded_image_data))

  # Assuming uploaded_image_data is binary image data in bytes
  image_array = np.frombuffer(uploaded_image_data, np.uint8)
  image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

  # Convert the image to RGB format
  uploaded_image_data = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
upload_button.observe(handle_upload_student, names='value')

In [ ]:
def handle_capture_student(button):
  global uploaded_image_data
  # start streaming video from webcam
  video_stream()
  js_reply = video_frame("Capturing...", "")

  stop_video_stream()  # Stop the video stream after capturing the frame

  if not js_reply:
    with output:
      clear_output()
      print("Error capturing image!")

  # convert JS response to OpenCV Image
  uploaded_image_data = js_to_image(js_reply["img"])

  with output:
    clear_output()
    # Encode the captured frame as PNG
    _, buffer = cv2.imencode('.png', uploaded_image_data)

    # Convert the encoded image to bytes
    image_bytes = buffer.tobytes()
    display(widgets.Image(value=image_bytes))
capture_button.on_click(handle_capture_student)

In [ ]:
def calcular_edad(fecha_nacimiento):
    # Convierte la fecha de nacimiento en un objeto datetime
    fecha_nacimiento = datetime.strptime(fecha_nacimiento, '%Y-%m-%d')

    # Obtiene la fecha actual
    fecha_actual = datetime.now()

    # Calcula la diferencia en años entre la fecha actual y la fecha de nacimiento
    edad = fecha_actual.year - fecha_nacimiento.year

    # Ajusta la edad si la fecha actual aún no ha alcanzado el cumpleaños de la persona
    if fecha_actual.month < fecha_nacimiento.month or (fecha_actual.month == fecha_nacimiento.month and fecha_actual.day < fecha_nacimiento.day):
        edad -= 1

    return edad

In [ ]:
# Define a function to handle face registration
def handle_register_student(button):


  if uploaded_image_data is not None:

    registered_name = student_name_input.value
    registres_id = student_id_input.value

    # Detect faces in the frame
    face_locations = face_recognition.face_locations(uploaded_image_data)

    # If there is no face, raise a popup window
    if len(face_locations) == 0:
      with output:
        clear_output()
        print("No face detected in the image!")
        return

    # If there is more than one face, raise a popup window
    if len(face_locations) > 1:
      with output:
        clear_output()
        print("More than one face detected in the image!")
        return

    # Get the face encodings
    face_encodings = face_recognition.face_encodings(uploaded_image_data)

    # Check if the face is registered
    if check_if_face_registered(face_encodings[0]):
      with output:
        clear_output()
        print("Face already registered!")
        return

    with output:
      clear_output()
      print(f"Face registered for {registered_name}")
      registered_faces[registered_name] = face_encodings[0]

    # Nombre del archivo JSON
  archivo_bin = 'LoD_Student.bin'

  # Abrir el archivo JSON en modo lectura para cargar los datos existentes
  try:
      with open(archivo_bin, 'rb') as archivo:
          datos_existentes = pickle.load(archivo)
  except FileNotFoundError:
      # Si el archivo no existe, crea una lista vacía
      datos_existentes = []

  estudiante = Student(
    student_id_input.value,
    student_name_input.value,
    student_date_of_birth_input.value,
    student_nationallity_input.value,
    student_phone_input.value,
    student_email_input.value,
    student_address_input.value,
    student_career_dropdown.value
  )

  #Calcula las fechas para sacar edades
  fecha = str(student_date_of_birth_input.value)
  edad = calcular_edad(fecha)

  # Create a dictionary for teacher information
  student_info = {
      'ID': str(estudiante.id),
      'Name': str(estudiante.name),
      'Date of Birth': str(estudiante.date_of_birth),
      'Age': edad,
      'Nationallity': str(estudiante.nationallity),
      'Phone':str(estudiante.phone),
      'Email':str(estudiante.email),
      'Address':str(estudiante.address),
      'Career':str(estudiante.career),
      'Image': str(face_encodings[0])
  }

  # Agrega el nuevo diccionario a la lista de datos existente
  datos_existentes.append(student_info)
  student_list.append(student_info)


  # Abre el archivo JSON en modo escritura y guarda la lista actualizada
  with open(archivo_bin, 'wb') as archivo:
      pickle.dump(datos_existentes, archivo)  # El argumento indent hace que el archivo sea más legible
      print(f"Data successfully registered in {archivo_bin}")

register_button_student.on_click(handle_register_student)

In [ ]:
def add_student():
  # Create a container for the widgets
  container = widgets.VBox([student_name_input, student_id_input, student_date_of_birth_input, student_nationallity_input, student_phone_input, student_email_input, student_address_input, student_career_dropdown, widgets.HBox([upload_button, or_text, capture_button]), widgets.HBox([register_button_student, back_button]), output])

  html_label = widgets.HTML(
      value='<h1 style="color: white; font-size: 24px;">Add Student Details</h1>'
  )

  # Mostrar el widget en la celda de Jupyter Notebook
  display(html_label)
  display(container)

## Register Janitors

In [ ]:
class Janitor(People):
    def __init__(self, id, name, date_of_birth, nationallity, phone, email, address, shift):
      People.__init__(self, id, name, date_of_birth, nationallity, phone, email, address)
      self.shift = shift

In [ ]:
# Create a text input widget for the name
janitor_name_input = widgets.Text(
    placeholder='Enter the name',
    description='Name:',
    disabled=False
)

janitor_id_input = widgets.Text(
    placeholder='Enter the ID',
    description='ID:',
    disabled=False
)

janitor_id_button = widgets.Button(
    description="Generate ID",
    value = " "
)


janitor_date_of_birth_input = widgets.DatePicker(
  description='Date of birth:',
  disabled=False
)

janitor_nationallity_input = widgets.Text(
    placeholder='Enter the Nationallity:',
    description='Nationallity:',
    disabled=False
)

janitor_phone_input = widgets.Text(
    placeholder='Enter the Phone Number:',
    description='Phone:',
    disabled=False
)

janitor_email_input = widgets.Text(
    placeholder='Enter the Email:',
    description='Email:',
    disabled=False
)

janitor_address_input = widgets.Text(
    placeholder='Enter the Teacher Address:',
    description='Address:',
    disabled=False
)


opciones = ['Day', 'Night']
janitor_shift_dropdown = widgets.Dropdown(
  options=opciones,
  value=opciones[0],  # Valor inicial
  description='Shift:'
)

# Create a file upload widget
upload_button = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='Upload Image'
)

or_text = widgets.HTML(
    value=" or "
)


# Create a button to trigger face registration
register_button_janitor = widgets.Button(
    description='Register Face'
)
register_button_janitor.style.button_color = 'green'

exit_button = widgets.Button(
    description="Back to Menu"
)
exit_button.style.button_color = 'darkred'

capture_button = widgets.Button(
    description='Capture Image'
)

# Create an output widget to display the uploaded or captured face image
output = widgets.Output()

# Initialize variables to store the uploaded image and name
uploaded_image_data = None
registered_name = None

In [ ]:
# Define a function to handle image upload
def handle_upload_janitor(change):
  global uploaded_image_data
  uploaded_filename = list(upload_button.value.keys())[0]
  uploaded_image_data = upload_button.value[uploaded_filename]['content']

  with output:
      clear_output()
      display(widgets.Image(value=uploaded_image_data))

  # Assuming uploaded_image_data is binary image data in bytes
  image_array = np.frombuffer(uploaded_image_data, np.uint8)
  image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

  # Convert the image to RGB format
  uploaded_image_data = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
upload_button.observe(handle_upload_janitor, names='value')

In [ ]:
def handle_capture_janitor(button):
  global uploaded_image_data
  # start streaming video from webcam
  video_stream()
  js_reply = video_frame("Capturing...", "")

  stop_video_stream()  # Stop the video stream after capturing the frame

  if not js_reply:
    with output:
      clear_output()
      print("Error capturing image!")

  # convert JS response to OpenCV Image
  uploaded_image_data = js_to_image(js_reply["img"])

  with output:
    clear_output()
    # Encode the captured frame as PNG
    _, buffer = cv2.imencode('.png', uploaded_image_data)

    # Convert the encoded image to bytes
    image_bytes = buffer.tobytes()
    display(widgets.Image(value=image_bytes))

capture_button.on_click(handle_capture_janitor)

In [ ]:
# Define a function to handle face registration
def handle_register_janitor(button):

  if uploaded_image_data is not None:
    registered_name = janitor_name_input.value
    registres_id = janitor_id_input.value

    # Detect faces in the frame
    face_locations = face_recognition.face_locations(uploaded_image_data)

    # If there is no face, raise a popup window
    if len(face_locations) == 0:
      with output:
        clear_output()
        print("No face detected in the image!")
        return

    # If there is more than one face, raise a popup window
    if len(face_locations) > 1:
      with output:
        clear_output()
        print("More than one face detected in the image!")
        return

    # Get the face encodings
    face_encodings = face_recognition.face_encodings(uploaded_image_data)

    # Check if the face is registered
    if check_if_face_registered(face_encodings[0]):
      with output:
        clear_output()
        print("Face already registered!")
        return

    with output:
      clear_output()
      print(f"Face registered for {registered_name}")
      registered_faces[registered_name] = face_encodings[0]


    # Nombre del archivo JSON
  archivo_bin = 'LoD_Janitor.bin'

  # Abrir el archivo JSON en modo lectura para cargar los datos existentes
  try:
      with open(archivo_bin, 'rb') as archivo:
          datos_existentes = pickle.load(archivo)
  except FileNotFoundError:
      # Si el archivo no existe, crea una lista vacía
      datos_existentes = []

  micelaneo = Janitor(
      janitor_id_input.value,
      janitor_name_input.value,
      janitor_date_of_birth_input.value,
      janitor_nationallity_input.value,
      janitor_phone_input.value,
      janitor_email_input.value,
      janitor_address_input.value,
      janitor_shift_dropdown.value
  )

  # Create a dictionary for teacher information
  janitor_info = {
      'ID': str(micelaneo.id),
      'Name': str(micelaneo.name),
      'Date of Birth': str(micelaneo.date_of_birth),
      'Nationallity': str(micelaneo.nationallity),
      'Phone': str(micelaneo.phone),
      'Email': str(micelaneo.email),
      'Address': str(micelaneo.address),
      'Shift': str(micelaneo.shift),
      'Image': str(face_encodings[0])
  }

  # Agrega el nuevo diccionario a la lista de datos existente
  datos_existentes.append(janitor_info)

  # Abre el archivo JSON en modo escritura y guarda la lista actualizada
  with open(archivo_bin, 'wb') as archivo:
      pickle.dump(datos_existentes, archivo)  # El argumento indent hace que el archivo sea más legible
      print(f"Data successfully registered in {archivo_bin}")

register_button_janitor.on_click(handle_register_janitor)

In [ ]:
def add_janitor():
  # Create a container for the widgets
  container = widgets.VBox([janitor_name_input, janitor_id_input, janitor_date_of_birth_input, janitor_nationallity_input, janitor_phone_input, janitor_email_input, janitor_address_input, janitor_shift_dropdown, widgets.HBox([upload_button, or_text, capture_button]), widgets.HBox([register_button_janitor, back_button]), output])

  html_label = widgets.HTML(
      value='<h1 style="color: white; font-size: 24px;">Add Janitor Details</h1>'
  )

  # Mostrar el widget en la celda de Jupyter Notebook
  display(html_label)
  display(container)

#Register other details

## Register Classroom

In [ ]:
class Classroom():
  def __init__(self, name, id, schedule):
    self.name = name
    self.id = id
    self.schedule = schedule

In [ ]:
classroom_id_input = widgets.Text(
    placeholder='Enter the Classroom num',
    description='ID:',
    disabled=False
)

classroom_name_input = widgets.Text(
    placeholder='Enter the Name',
    description='Name:',
    disabled=False
)

classroom_schedule_input = widgets.Text(
    placeholder='Cleaning Schedule',
    description='Cleaning:',
    disabled=False
)

# Crear un botón
save_button_classroom = widgets.Button(
    description="Register Classroom"
  )
save_button_classroom.style.button_color = 'green'

# Create an output widget to display the uploaded or captured face image
output = widgets.Output()

In [ ]:
# Wait for user input and store data
id_classroom_input = classroom_id_input.value
name_classroom_input = classroom_name_input.value
schedule_classroom_input = classroom_schedule_input.value

# Función que se ejecutará cuando se haga clic en el botón
def boton_clickeado(boton):

  lista_micelaneo = []
  #lista_micelaneo.append(janitor_list)

  # Nombre del archivo JSON
  archivo_bin = 'LoD_Classroom.bin'

  # Abrir el archivo JSON en modo lectura para cargar los datos existentes
  try:
      with open(archivo_bin, 'rb') as archivo:
          datos_existentes = pickle.load(archivo)
  except FileNotFoundError:
      # Si el archivo no existe, crea una lista vacía
      datos_existentes = []

  aula = Classroom(
      classroom_id_input.value,
      classroom_name_input.value,
      classroom_schedule_input.value
  )

  # Datos que deseas agregar al archivo JSON (nuevo diccionario)
  classroom_info = {
      'ID': str(aula.id),
      'Name': str(aula.name),
      'Schedule': str(aula.schedule),
      'list of janitors': lista_micelaneo
  }

  # Agrega el nuevo diccionario a la lista de datos existente
  datos_existentes.append(classroom_info)

  # Abre el archivo JSON en modo escritura y guarda la lista actualizada
  with open(archivo_bin, 'wb') as archivo:
      pickle.dump(datos_existentes, archivo)  # El argumento indent hace que el archivo sea más legible
      print(f"Data successfully registered in {archivo_bin}")

save_button_classroom.on_click(boton_clickeado)

In [ ]:
html_label_class = widgets.HTML(
    value='<h1 style="color: white; font-size: 24px;">Add Classroom Details</h1>'
)

def add_classroom():
  # Mostrar el widget en la celda de Jupyter Notebook
  display(html_label_class)
  display(classroom_id_input, classroom_name_input, classroom_schedule_input, widgets.HBox([save_button_classroom, back_button]))

##Register Course

In [ ]:
class Course():
  def __init__(self, name, id, period, career, schedule):
    self.name = name
    self.id = id
    self.period = period
    self.career = career
    self.schedule = schedule

In [ ]:
# Create a text input widget for the name
course_name_input = widgets.Text(
    placeholder='Enter the name',
    description='Name:',
    disabled=False
)

course_id_input = widgets.Text(
    placeholder='Enter the ID',
    description='ID:',
    disabled=False
)

course_period_input = widgets.Text(
    placeholder='Enter the Period',
    description='Period:',
    disabled=False
)

opciones = [
            "Software Engineering",
            "Industrial Production Engineering",
            "Engineering in Agronomy",
            "Business Administration",
            "Electronics Engineering",
            "Computer Engineering",
            "Sustainable Rural Tourism Management",
            "Tourism Sustainability Management",
            ]
course_career_dropdown = widgets.Dropdown(
  options=opciones,
  description='Career:',
  value=opciones[0]
)

course_schedule_input = widgets.Text(
    placeholder='Enter the Schedule',
    description='Schedule:',
    disabled=False
)

# Crear un botón
save_button = widgets.Button(
    description="Register Course"
)
save_button.style.button_color = 'green'
exit_button = widgets.Button(
    description="Back to Menu"
)
exit_button.style.button_color = 'darkred'

# Create an output widget to display the uploaded or captured face image
output = widgets.Output()

In [ ]:
# Función que se ejecutará cuando se haga clic en el botón
def register_button(boton):


  # Nombre del archivo JSON
  archivo_bin = 'LoD_Course.bin'

  # Abrir el archivo JSON en modo lectura para cargar los datos existentes
  try:
      with open(archivo_bin, 'rb') as archivo:
          datos_existentes = pickle.load(archivo)
  except FileNotFoundError:
      # Si el archivo no existe, crea una lista vacía
      datos_existentes = []

  cursos = Course (
    course_name_input.value,
    course_id_input.value,
    course_period_input.value,
    course_career_dropdown.value,
    course_schedule_input.value
  )

  # Create a dictionary for teacher information
  course_info = {
      'Name': str(cursos.name),
      'ID': str(cursos.id),
      'Period': str(cursos.period),
      'Career': str(cursos.career),
      'Schedule': str(cursos.schedule),
      'List of Students': student_list,
      'List of Teachers': teacher_list,
  }

  # Agrega el nuevo diccionario a la lista de datos existente
  datos_existentes.append(course_info)

  # Abre el archivo JSON en modo escritura y guarda la lista actualizada
  with open(archivo_bin, 'wb') as archivo:
      pickle.dump(datos_existentes, archivo)  # El argumento indent hace que el archivo sea más legible
      print(f"Data successfully registered in {archivo_bin}")
save_button.on_click(register_button)

In [ ]:
html_label_course = widgets.HTML(
    value='<h1 style="color: white; font-size: 24px;">Add Course Details</h1>'
)

def add_course():
  # Mostrar el widget en la celda de Jupyter Notebook
  display(html_label_course)
  display(course_name_input, course_id_input, course_period_input, course_career_dropdown, course_schedule_input, widgets.HBox([save_button, back_button]))
add_course()

# Data Adder

In [ ]:
def teacher_button_add(b):
  clear_output()
  add_teacher()
def student_button_add(b):
  clear_output()
  add_student()
def janitor_button_add(b):
  clear_output()
  add_janitor()
def classroom_button_add(b):
  clear_output()
  add_classroom()
def course_button_add(b):
  clear_output()
  add_course()
def menu_button_add(b):
  clear_output()
  main()

# Crear un botón
add_teacher_button = widgets.Button(description="Add Teacher")
add_student_button = widgets.Button(description="Add Student")
add_janitor_button = widgets.Button(description="Add Janitor")
add_classroom_button = widgets.Button(description="Add Classroom")
add_course_button = widgets.Button(description = "Add Course")
add_back_menu_button = widgets.Button(description = "Back Menu")
add_back_menu_button.style.button_color = 'darkred'

# Asignar la función de clic al botón
add_teacher_button.on_click(teacher_button_add)
add_student_button.on_click(student_button_add)
add_janitor_button.on_click(janitor_button_add)
add_classroom_button.on_click(classroom_button_add)
add_course_button.on_click(course_button_add)
add_back_menu_button.on_click(menu_button_add)

def dataAdder():
  # Mostrar el botón y el área de salida
  container = widgets.HBox([add_teacher_button, add_student_button, add_janitor_button, add_classroom_button, add_course_button, add_back_menu_button])

  html_label = widgets.HTML(
      value='<h1 style="color: white; font-size: 24px;">Data Adder Menu</h1>'
  )

  # Mostrar el widget en la celda de Jupyter Notebook
  display(html_label)
  display(container)

#Data Reader

In [ ]:
back_reader_button = widgets.Button(description = "Back Menu")
back_reader_button.style.button_color = 'darkred'
def back_data_reader(b):
  clear_output()
  dataReader()
back_reader_button.on_click(back_data_reader)

In [ ]:
def show_teacher():
  # Abre el archivo JSON en modo lectura
  with open('LoD_Teacher.bin', 'rb') as archivo:
      # Carga los datos desde el archivo JSON
      datos = pickle.load(archivo)

  i = 1
  # Itera a través de la lista de diccionarios
  for element in datos:

      html_label = widgets.HTML(
      value=(f'<h1 style="color: white; font-size: 18px;">Teacher {i}:</h1>')
      )

      id = element['ID']
      name = element['Name']
      dateofbirth = element['Date of Birth']
      nationallity = element['Nationallity']
      phone = element['Phone']
      email = element['Email']
      address = element['Address']
      speciality = element['Speciality']
      image = element['Image']
      i += 1

      display(html_label)
      print(f'Name: {name}','\n')
      print(f'ID: {id}','\n')
      print(f'Date of birth: {dateofbirth}','\n')
      print(f'Nationallity: {nationallity}','\n')
      print(f'Phone: {phone}','\n')
      print(f'Email: {email}','\n')
      print(f'Address: {address}','\n')
      print(f'Specility: {speciality}','\n')
      print(f'Image: {image}','\n')
  display(back_reader_button)

In [ ]:
def show_student():
  # Abre el archivo JSON en modo lectura
  with open('LoD_Student.bin', 'rb') as archivo:
      # Carga los datos desde el archivo JSON
      datos = pickle.load(archivo)

  i = 1
  # Itera a través de la lista de diccionarios
  for element in datos:

      html_label = widgets.HTML(
      value=(f'<h1 style="color: white; font-size: 18px;">Student {i}:</h1>')
      )

      id = element['ID']
      name = element['Name']
      dateofbirth = element['Date of Birth']
      nationallity = element['Nationallity']
      phone = element['Phone']
      email = element['Email']
      address = element['Address']
      career = element['Career']
      img = element['Image']
      i += 1

      display(html_label)
      print(f'Name: {name}','\n')
      print(f'ID: {id}','\n')
      print(f'Date of birth: {dateofbirth}','\n')
      print(f'Nationallity: {nationallity}','\n')
      print(f'Phone: {phone}','\n')
      print(f'Email: {email}','\n')
      print(f'Address: {address}','\n')
      print(f'Career: {career}','\n')
      print(f'Image: {img}','\n')
  display(back_reader_button)

In [ ]:
def bubble_sort():
  with open('LoD_Student.bin', 'rb') as archivo:
    # Carga los datos desde el archivo JSON
    datos = pickle.load(archivo)
    n = len(datos)

    # Recorremos la lista
    for i in range(n):
        # Marcador para saber si se realizó algún intercambio
        intercambio = False

        # Últimos 'i' elementos ya están en su lugar
        for j in range(0, n - i - 1):
            # Comparamos el elemento actual con el siguiente
            if datos[j]["Age"] > datos[j + 1]["Age"]:
                # Intercambiamos los elementos
                datos[j], datos[j + 1] = datos[j + 1], datos[j]
                intercambio = True

        # Si no se realizó ningún intercambio en esta pasada, la lista está ordenada
        if not intercambio:
            break

    i = 1
    # Itera a través de la lista de diccionarios
    html_label1 = widgets.HTML(
    value=(f'<h1 style="color: white; font-size: 25px;">Students:</h1>')
    )
    display(html_label1)
    # Itera a través de la lista de diccionarios
    for element in datos:


        html_label2 = widgets.HTML(
        value=(f'<h1 style="color: white; font-size: 18px;">Students {i}:</h1>')
        )

        id = element['ID']
        name = element['Name']
        dateofbirth = element['Date of Birth']
        age = element['Age']
        nationallity = element['Nationallity']
        phone = element['Phone']
        email = element['Email']
        address = element['Address']
        career = element['Career']
        img = element['Image']
        i += 1

        display(html_label2)
        print(f'Name: {name}','\n')
        print(f'ID: {id}','\n')
        print(f'Date of birth: {dateofbirth}','\n')
        print(f'Age: {age}','\n')
        print(f'Nationallity: {nationallity}','\n')
        print(f'Phone: {phone}','\n')
        print(f'Email: {email}','\n')
        print(f'Address: {address}','\n')
        print(f'Career: {career}','\n')
        print(f'Image: {img}','\n')
    display(back_reader_button)

In [ ]:
def insertion_sort_descending():
  with open('LoD_Student.bin', 'rb') as archivo:
    # Carga los datos desde el archivo JSON
    datos = pickle.load(archivo)
    n = len(datos)

    for i in range(1, n):
        dic = datos[i]
        j = i - 1

        # Mover elementos de arr[0..i-1] que son mayores que key
        while j >= 0 and dic["Age"] > datos[j]["Age"]:
            datos[j + 1] = datos[j]
            j -= 1

        datos[j + 1] = dic

    #print("Lista ordenada de forma descendente:")
    #print(datos[0]["Age"])


    i = 1
    # Itera a través de la lista de diccionarios
    html_label1 = widgets.HTML(
    value=(f'<h1 style="color: white; font-size: 25px;">Students:</h1>')
    )
    display(html_label1)

    for element in datos:



        html_label2 = widgets.HTML(
        value=(f'<h1 style="color: white; font-size: 18px;">Student {i}:</h1>')
        )

        id = element['ID']
        name = element['Name']
        dateofbirth = element['Date of Birth']
        age = element['Age']
        nationallity = element['Nationallity']
        phone = element['Phone']
        email = element['Email']
        address = element['Address']
        career = element['Career']
        img = element['Image']
        i += 1


        display(html_label2)
        print(f'Name: {name}','\n')
        print(f'ID: {id}','\n')
        print(f'Date of birth: {dateofbirth}','\n')
        print(f'Age: {age}','\n')
        print(f'Nationallity: {nationallity}','\n')
        print(f'Phone: {phone}','\n')
        print(f'Email: {email}','\n')
        print(f'Address: {address}','\n')
        print(f'Career: {career}','\n')
        print(f'Image: {img}','\n')
    display(back_reader_button)

In [ ]:
def show_janitor():
  # Abre el archivo JSON en modo lectura
  with open('LoD_Janitor.bin', 'rb') as archivo:
      # Carga los datos desde el archivo JSON
      datos = pickle.load(archivo)

  i = 1
  # Itera a través de la lista de diccionarios
  for element in datos:

      html_label = widgets.HTML(
      value=(f'<h1 style="color: white; font-size: 18px;">Janitor {i}:</h1>')
      )

      id = element['ID']
      name = element['Name']
      dateofbirth = element['Date of Birth']
      nationallity = element['Nationallity']
      phone = element['Phone']
      email = element['Email']
      address = element['Address']
      shift = element['Shift']
      image = element['Image']
      i += 1

      display(html_label)
      print(f'Name: {name}','\n')
      print(f'ID: {id}','\n')
      print(f'Date of birth: {dateofbirth}','\n')
      print(f'Nationallity: {nationallity}','\n')
      print(f'Phone: {phone}','\n')
      print(f'Email: {email}','\n')
      print(f'Address: {address}','\n')
      print(f'Career: {shift}','\n')
      print(f'Image: {image}','\n')
  display(back_reader_button)

In [ ]:
def show_classroom():
  # Abre el archivo JSON en modo lectura
  with open('LoD_Classroom.bin', 'rb') as archivo:
      # Carga los datos desde el archivo JSON
      datos = pickle.load(archivo)

  i = 1
  # Itera a través de la lista de diccionarios
  for element in datos:

      html_label = widgets.HTML(
      value=(f'<h1 style="color: white; font-size: 18px;">Classroom {i}:</h1>')
      )

      id = element['ID']
      name = element['Name']
      schedule = element['Schedule']
      list_of_janitor = element['list of janitors']
      i += 1

      display(html_label)
      print(f'Name: {name}','\n')
      print(f'ID: {id}','\n')
      print(f'Date of birth: {schedule}','\n')
      print(f'Nationallity: {list_of_janitor}','\n')
  display(back_reader_button)

In [ ]:
def show_course():
  # Abre el archivo JSON en modo lectura
  with open('LoD_Course.bin', 'rb') as archivo:
      # Carga los datos desde el archivo JSON
      datos = pickle.load(archivo)

  i = 1
  # Itera a través de la lista de diccionarios
  for element in datos:

      html_label = widgets.HTML(
      value=(f'<h1 style="color: white; font-size: 18px;">Course {i}:</h1>')
      )

      id = element['ID']
      name = element['Name']
      period = element['Period']
      career = element['Career']
      schedule = element['Schedule']
      lista_te = element['List of Teachers']
      lista_es = element['List of Students']
      i += 1



      display(html_label)
      print(f'Name: {name}','\n')
      print(f'ID: {id}','\n')
      print(f'Period: {period}','\n')
      print(f'Career: {career}','\n')
      print(f'Schedule: {schedule}','\n')
      print(f'Teachers: {lista_te}','\n')
      print(f'Students: {lista_es}','\n')
  display(back_reader_button)

In [ ]:
def show_teacher_butto(b):
  clear_output()
  show_teacher()
def show_student_butto(b):
  clear_output()
  show_student()
def show_janitor_butto(b):
  clear_output()
  show_janitor()
def show_classroom_butto(b):
  clear_output()
  show_classroom()
def show_course_butto(b):
  clear_output()
  show_course()
def show_student_des_bu(b):
  clear_output()
  insertion_sort_descending()
def show_student_as_bu(b):
  clear_output()
  bubble_sort()
def menu_button(b):
  clear_output()
  main()


# Crear un botón
show_teacher_button = widgets.Button(description="Show Teachers")
show_student_button = widgets.Button(description="Show Students")
show_janitor_button = widgets.Button(description="Show Janitors")
show_classroom_button = widgets.Button(description="Show Classrooms")
show_course_button = widgets.Button(description = "show Courses")
show_student_des_button = widgets.Button(description="Students Descending")
show_student_as_button = widgets.Button(description="Students Ascending")
show_back_menu_button = widgets.Button(description = "Back Menu")
show_back_menu_button.style.button_color = 'darkred'

# Asignar la función de clic al botón
show_teacher_button.on_click(show_teacher_butto)
show_student_button.on_click(show_student_butto)
show_janitor_button.on_click(show_janitor_butto)
show_classroom_button.on_click(show_classroom_butto)
show_course_button.on_click(show_course_butto)
show_student_des_button.on_click(show_student_des_bu)
show_student_as_button.on_click(show_student_as_bu)
show_back_menu_button.on_click(menu_button)

def dataReader():
  # Mostrar el botón y el área de salida
  container = widgets.HBox([widgets.VBox([show_teacher_button, show_student_des_button]), widgets.VBox([show_student_button, show_student_as_button]), show_janitor_button, show_classroom_button, show_course_button, show_back_menu_button, ])


  html_label = widgets.HTML(
      value='<h1 style="color: white; font-size: 24px;">Data Reader Menu</h1>'
  )
  # Mostrar el widget en la celda de Jupyter Notebook
  display(html_label)
  display(container)

#Recognize Faces

In [ ]:
def facial_recognition():
  # start streaming video from webcam
  video_stream()
  # label for video
  label_html = 'Capturing...'
  bbox = ''

  while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
      break

    # convert JS response to OpenCV Image
    frame = js_to_image(js_reply["img"])

    # Get the face locations from the frame
    face_locations = face_recognition.face_locations(frame)

    # For each face in the frame
    for (top, right, bottom, left) in face_locations:
        # Draw a rectangle around the face
        cv2.rectangle(
            frame,
            (left, top),
            (right, bottom),
            (0, 0, 255),
            2,
        )
        # Get the face encoding of the face
        face_encoding = face_recognition.face_encodings(frame, [(top, right, bottom, left)])[0]

        # Compare the face encoding to the faces in the database
        matches = face_recognition.compare_faces(list(registered_faces.values()), face_encoding)

        # If there is a match
        if True in matches:
            # Get the index of the match
            match_index = matches.index(True)

            # Get the name of the match
            match_name = list(registered_faces.keys())[match_index]

            # Draw the name of the match on the frame
            cv2.putText(
                frame,
                match_name,
                (left, top - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 0, 255),
                2,
                cv2.LINE_AA,
            )
        else:
            # Draw unknown on the frame
            cv2.putText(
                frame,
                "Unknown",
                (left, top - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 0, 255),
                2,
                cv2.LINE_AA,
            )

    # Show image
    plt.imshow(frame)
    plt.show()

#Principal Menu

In [ ]:
# Crear un botón
add_button = widgets.Button(description="Add Information")
ask_button = widgets.Button(description="Ask Information")
facial_recognition_button = widgets.Button(description="Face Recognition")
ask_button = widgets.Button(description="Ask Information")
exit_button = widgets.Button(description="EXIT")
exit_button.style.button_color = 'darkred'

  # Función que se ejecutará cuando se haga clic en el botón
def add_information(b):
  clear_output()
  dataAdder()
def ask_information(b):
  clear_output()
  dataReader()
def show_recognition(b):
  clear_output()
  facial_recognition()
def exit(b):
  clear_output()
  display(finish_label)


# Asignar la función de clic al botón
add_button.on_click(add_information)
ask_button.on_click(ask_information)
facial_recognition_button.on_click(show_recognition)
exit_button.on_click(exit)

html_label = widgets.HTML(
    value='<h1 style="color: white; font-size: 24px;">Welcome To the System</h1>'
)

finish_label = widgets.HTML(
    value='<h1 style="color: white; font-size: 24px;">System Finished</h1>'
)

def main():
  # Mostrar el widget en la celda de Jupyter Notebook
  display(html_label)

  # Mostrar el botón y el área de salida
  container = widgets.HBox([add_button, ask_button, facial_recognition_button, exit_button])
  display(container)

main()

HTML(value='<h1 style="color: white; font-size: 24px;">Welcome To the System</h1>')